In [1]:
import sqlite3
import pandas as pd

# Step 2: Load CSV Files
content_path = "Content (1).csv"
reaction_path = "Reactions (1).csv"
reaction_types_path = "ReactionTypes (1).csv"

content = pd.read_csv(content_path)
reaction = pd.read_csv(reaction_path)
reaction_types = pd.read_csv(reaction_types_path)

# Step 3: Create SQLite Database and Load Data
conn = sqlite3.connect('user_personas.db')
cursor = conn.cursor()

In [2]:
# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS Content (
    ContentID INTEGER,
    UserID INTEGER,
    Type TEXT,
    Category TEXT,
    URL TEXT
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Reaction (
    ContentID INTEGER,
    UserID INTEGER,
    Type TEXT,
    Datetime TEXT
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS ReactionTypes (
    Type TEXT,
    Sentiment TEXT,
    Score INTEGER
);
''')

# Load data into SQLite
def load_to_sqlite(table_name, dataframe):
    dataframe.to_sql(table_name, conn, if_exists='replace', index=False)

load_to_sqlite("Content", content)
load_to_sqlite("Reaction", reaction)
load_to_sqlite("ReactionTypes", reaction_types)

In [3]:
# Remove duplicates
content = content.drop_duplicates(subset="ContentID")

# Drop rows with missing values in critical columns
content = content.dropna(subset=["ContentID", "Category", "Type"])



In [4]:
# Remove duplicates
reaction = reaction.drop_duplicates()

# Drop rows with missing values in critical columns
reaction = reaction.dropna(subset=["ContentID", "Datetime"])

# Convert `Datetime` to datetime format
reaction["Datetime"] = pd.to_datetime(reaction["Datetime"])

# Filter rows where `ContentID` is not in `Content.csv`
reaction = reaction[reaction["ContentID"].isin(content["ContentID"])]


C:\Users\PC\AppData\Local\Temp\ipykernel_29604\3183786796.py:8: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  reaction["Datetime"] = pd.to_datetime(reaction["Datetime"])


In [5]:
# Remove duplicates
reaction_types = reaction_types.drop_duplicates()

# Drop rows with missing values in critical columns
reaction_types = reaction_types.dropna(subset=["Type", "Sentiment"])

# Ensure sentiment values are consistent
valid_sentiments = ["positive", "negative", "neutral"]
reaction_types = reaction_types[reaction_types["Sentiment"].isin(valid_sentiments)]


In [6]:

# 1. Top 5 Categories by Popularity
query_top_categories = """
SELECT 
    C.Category, 
    SUM(RT.Score) AS TotalPopularity
FROM 
    Reaction R
JOIN 
    Content C ON R.ContentID = C.ContentID
JOIN 
    ReactionTypes RT ON R.Type = RT.Type
GROUP BY 
    C.Category
ORDER BY 
    TotalPopularity DESC
LIMIT 5;
"""
df_top_categories = pd.read_sql(query_top_categories, conn)
print("Top 5 Categories by Popularity:\n", df_top_categories)

Top 5 Categories by Popularity:
          Category  TotalPopularity
0         animals            69548
1  healthy eating            69067
2      technology            67472
3         science            66043
4         cooking            63982


In [7]:
# 2. Sentiment Analysis by Category
query_sentiment_analysis = """
SELECT 
    C.Category, 
    RT.Sentiment, 
    COUNT(RT.Sentiment) AS SentimentCount
FROM 
    Reaction R
JOIN 
    Content C ON R.ContentID = C.ContentID
JOIN 
    ReactionTypes RT ON R.Type = RT.Type
GROUP BY 
    C.Category, RT.Sentiment;
"""
df_sentiment_analysis = pd.read_sql(query_sentiment_analysis, conn)
print("Sentiment Analysis by Category:\n", df_sentiment_analysis)

Sentiment Analysis by Category:
       Category Sentiment  SentimentCount
0    "animals"  negative              11
1    "animals"   neutral               5
2    "animals"  positive              24
3    "cooking"  negative              10
4    "cooking"   neutral               1
..         ...       ...             ...
110     travel   neutral             192
111     travel  positive             904
112   veganism  negative             372
113   veganism   neutral             162
114   veganism  positive             666

[115 rows x 3 columns]


In [8]:
# 3. Content-Type Analysis
query_content_type_analysis = """
SELECT 
    C.Category, 
    C.Type AS ContentType, 
    SUM(RT.Score) AS TotalPopularity
FROM 
    Reaction R
JOIN 
    Content C ON R.ContentID = C.ContentID
JOIN 
    ReactionTypes RT ON R.Type = RT.Type
GROUP BY 
    C.Category, C.Type
ORDER BY 
    TotalPopularity DESC;
"""
df_content_type_analysis = pd.read_sql(query_content_type_analysis, conn)
print("Content-Type Analysis:\n", df_content_type_analysis)

Content-Type Analysis:
            Category ContentType  TotalPopularity
0           animals       photo            24464
1        technology       audio            23160
2           animals       audio            21628
3         education       photo            20889
4    healthy eating       audio            20712
..              ...         ...              ...
98   Healthy Eating       video              272
99           Soccer         GIF              245
100      Technology       video              217
101       Education         GIF              152
102          "food"       audio               50

[103 rows x 3 columns]


In [9]:
# 4. Engagement Trends
query_engagement_trends = """
SELECT 
    strftime('%Y-%m', R.Datetime) AS YearMonth, 
    COUNT(DISTINCT R.UserID) AS UniqueUsers, 
    COUNT(*) AS TotalReactions
FROM 
    Reaction R
GROUP BY 
    YearMonth
ORDER BY 
    YearMonth;
"""
df_engagement_trends = pd.read_sql(query_engagement_trends, conn)
print("Engagement Trends:\n", df_engagement_trends)

Engagement Trends:
   YearMonth  UniqueUsers  TotalReactions
0      None          500           25553


In [10]:
# 5. User Segmentation
query_user_segmentation = """
SELECT 
    R.UserID, 
    COUNT(R.ContentID) AS TotalReactions, 
    SUM(RT.Score) AS TotalScore
FROM 
    Reaction R
JOIN 
    ReactionTypes RT ON R.Type = RT.Type
GROUP BY 
    R.UserID
ORDER BY 
    TotalReactions DESC;
"""
df_user_segmentation = pd.read_sql(query_user_segmentation, conn)
print("User Segmentation:\n", df_user_segmentation)

User Segmentation:
                                    UserID  TotalReactions  TotalScore
0                                    None            2039       80163
1    c76c3393-88e2-47b0-ac37-dc4f2053f5a5              65        2757
2    68724f58-bc4d-4ab0-a4e1-60cdd5e95e7d              65        2762
3    0871bb31-3d6e-4e4c-ab19-95a262cac0d4              63        2287
4    d1a89d23-7d17-4949-9e1a-637317141f3d              62        2820
..                                    ...             ...         ...
496  3663e3e6-3d5c-4ed9-a6af-1e680ec5f34b              31        1082
497  e57c1d53-11ce-4df6-bb4b-85647776fd6d              30        1208
498  b4a6b3ac-b6af-4525-8d59-7afc00ff279d              30         993
499  90898216-e580-46c0-8e79-f2df84a9676d              30        1437
500  a710ab29-b72a-42c8-a79b-42e63d4a8bfd              29        1124

[501 rows x 3 columns]


In [11]:
# Step 6: Aggregate Data with SQL Queries
query = '''
SELECT 
    r.UserID,
    SUM(CASE WHEN rt.Sentiment = 'positive' THEN 1 ELSE 0 END) AS PositiveReactions,
    SUM(CASE WHEN rt.Sentiment = 'negative' THEN 1 ELSE 0 END) AS NegativeReactions,
    SUM(CASE WHEN rt.Sentiment = 'neutral' THEN 1 ELSE 0 END) AS NeutralReactions,
    COUNT(r.ContentID) AS TotalReactions,
    COUNT(DISTINCT DATE(r.Datetime)) AS ActiveDays
FROM 
    Reaction r
JOIN 
    ReactionTypes rt ON r.Type = rt.Type
GROUP BY 
    r.UserID;
'''

user_reactions = pd.read_sql_query(query, conn)

In [12]:
# Step 4: Perform SQL Queries for Aggregation
query = """
WITH ReactionAggregated AS (
    SELECT 
        r.ContentID,
        c.Category,
        rt.Sentiment,
        SUM(rt.Score) AS PopularityScore
    FROM Reaction r
    JOIN Content c ON r.ContentID = c.ContentID
    JOIN ReactionTypes rt ON r.Type = rt.Type
    GROUP BY r.ContentID, c.Category, rt.Sentiment
)
SELECT 
    Category,
    Sentiment,
    SUM(PopularityScore) AS TotalPopularity
FROM ReactionAggregated
GROUP BY Category, Sentiment
ORDER BY TotalPopularity DESC;
"""

aggregated_df = pd.read_sql_query(query, conn)

# Save the aggregated result to a CSV file
aggregated_csv_file = "aggregatedreaction.csv"
aggregated_df.to_csv(aggregated_csv_file, index=False)

print(f"Aggregated data saved to {aggregated_csv_file}")

# Step 5: Generate User Personas
query_user_personas = """
WITH ReactionWithScores AS (
    SELECT 
        r.UserID,
        r.ContentID,
        rt.Type AS ReactionType,
        rt.Score
    FROM Reaction r
    JOIN ReactionTypes rt ON r.Type = rt.Type
),
UserEngagement AS (
    SELECT 
        UserID,
        COUNT(DISTINCT ContentID) AS TotalReactions,
        AVG(Score) AS AvgReactionScore
    FROM ReactionWithScores
    GROUP BY UserID
)
SELECT 
    UserID,
    TotalReactions,
    AvgReactionScore,
    CASE 
        WHEN TotalReactions > 50 THEN 'Highly Active'
        WHEN TotalReactions BETWEEN 10 AND 50 THEN 'Moderately Active'
        ELSE 'Occasional User'
    END AS EngagementLevel
FROM UserEngagement;
"""

user_personas_df = pd.read_sql_query(query_user_personas, conn)

# Save user personas to a CSV file
user_personas_csv_file = "user_personas.csv"
user_personas_df.to_csv(user_personas_csv_file, index=False)

print(f"User personas data saved to {user_personas_csv_file}")

# Close the SQLite connection
conn.close()

Aggregated data saved to aggregatedreaction.csv
User personas data saved to user_personas.csv
